In [ ]:
import pandas as pd
import geopandas as gpd
import pydeck as pdk
import matplotlib.pyplot as plt
import numpy as np
import folium
from census import Census
from us import states

# Get a US Census API Key [here](https://api.census.gov/data/key_signup.html) 

Copy-paste your API Key when prompted when running the cell below.

In [ ]:
import os
from getpass import getpass

# Try to read from env first, otherwise prompt you
CENSUS_API_KEY = os.getenv("CENSUS_API_KEY") or getpass("Enter your US Census API key: ")

os.environ["CENSUS_API_KEY"] = CENSUS_API_KEY

print("Key loaded, length:", len(CENSUS_API_KEY), "characters")

# Choose table to view data for:

**B01003_001E**: Total population

**B19013_001E**: Median household income

In [ ]:
table_name = "B19013_001E"  # Total population

In [ ]:
c = Census(CENSUS_API_KEY, year=2022)

# List of Bay-Area county FIPS:
bay_fips = ["001","013","041","055","075","081","085","095","097"]

# Pull Total Population (B01003_001E) by tract
pop = []
for county in bay_fips:
    pop += c.acs5.state_county_tract(
        (table_name,"NAME"),
        states.CA.fips,
        county,
        Census.ALL
    )

df = pd.DataFrame(pop)
df["GEOID"] = df.state + df.county + df.tract
df = df[["GEOID",table_name]]
population = df.rename(columns={table_name:"total_pop"})

df.head()

In [ ]:
# Load Bay-Area tracts shapefile
path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/TIGER Line 2022 Tract/tl_2022_06_tract.shp"
tracts = gpd.read_file(path)[["GEOID","geometry"]]
bay_tracts = tracts[tracts.GEOID.str[:5].isin({"06001","06013","06041",
                                             "06055","06075","06081",
                                             "06085","06095","06097"})]

# Merge population → GeoDataFrame
gdf = bay_tracts.merge(population, on="GEOID", how="left").fillna(0)

# Plot choropleth
fig, ax = plt.subplots(1,1, figsize=(10,10))
gdf.plot(
    column="total_pop",
    cmap="viridis",
    legend=True,
    legend_kwds={"label":"Total Population","fmt":"{:,.0f}"},
    linewidth=0.1,
    edgecolor="gray",
    ax=ax
)
ax.set_title("Bay Area Total Population by Census Tract (ACS 5-Year)")
ax.axis("off")
plt.show()

In [ ]:
# Define your view over the Bay Area
geojson = gdf.__geo_interface__
coords = np.vstack(gdf.geometry.centroid.apply(lambda p: (p.y, p.x)))
view_state = pdk.ViewState(
    latitude=coords[:,0].mean(),
    longitude=coords[:,1].mean(),
    zoom=9,
    pitch=0
)

# Center map on Bay Area
center = [view_state.latitude, view_state.longitude]
m = folium.Map(location=center, zoom_start=9, tiles=None)

# Add Google Streets as your basemap
folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Streets",
    control=False
).add_to(m)

# Add choropleth layer with legend
folium.Choropleth(
    geo_data=geojson,
    data=gdf,
    columns=["GEOID","total_pop"],
    key_on="feature.properties.GEOID",
    fill_color="YlOrBr",
    bins=8,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Population",
).add_to(m)

folium.LayerControl().add_to(m)
m

In [ ]:
# List of Bay-Area county FIPS:
bay_fips = ["001","013","041","055","075","081","085","095","097"]

# Bay Area county FIPS codes (without the state “06” prefix)
bay_fips = ["001", "013", "041", "055", "075", "081", "085", "095", "097"]

# Fetch ACS 5-Year total population (B01003) for each Bay Area county
records = []
for fips in bay_fips:
    recs = c.acs5.state_county(
        ("B01003_001E",),
        states.CA.fips, fips
    )
    records.extend(recs)

pop_df = pd.DataFrame(records)
pop_df["GEOID"] = pop_df.state + pop_df.county              # e.g. "06" + "001" -> "06001"
pop_df = pop_df[["GEOID", "B01003_001E"]].rename(
    columns={"B01003_001E": "total_population"}
)
pop_df.head()

In [ ]:
# Filter to the nine Bay Area counties
path = "/Users/dsong/Library/CloudStorage/OneDrive-UniversityofIllinois-Urbana/Research/UROP 2025 - UAM/Demand Analysis/TIGER Line 2022 County/tl_2022_us_county.shp"
counties = gpd.read_file(path)[["GEOID", "geometry"]]
bay_geoids = ["06" + f for f in bay_fips]
counties_bay = counties[counties.GEOID.isin(bay_geoids)].copy()

# Merge population data onto the geometries
gdf = counties_bay.merge(pop_df, on="GEOID", how="left").fillna(0)

# Plot a choropleth in Matplotlib
fig, ax = plt.subplots(figsize=(8, 8))
gdf.plot(
    column="total_population",
    cmap="YlGnBu",
    legend=True,
    legend_kwds={"label": "Total Population", "fmt": "{:,.0f}"},
    edgecolor="black",
    linewidth=0.4,
    ax=ax
)
ax.set_title("San Francisco Bay Area Total Population by County (ACS 5-Year)")
ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# Define view over the Bay Area
geojson = gdf.__geo_interface__
coords = np.vstack(gdf.geometry.centroid.apply(lambda p: (p.y, p.x)))
view_state = pdk.ViewState(
    latitude=coords[:,0].mean(),
    longitude=coords[:,1].mean(),
    zoom=9,
    pitch=0
)

# Center map on Bay Area
center = [view_state.latitude, view_state.longitude]
m = folium.Map(location=center, zoom_start=9, tiles=None)

# Add Google Streets as your basemap
folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Streets",
    control=False
).add_to(m)

# Add choropleth layer with legend
folium.Choropleth(
    geo_data=geojson,
    data=gdf,
    columns=["GEOID","total_population"],
    key_on="feature.properties.GEOID",
    fill_color="YlOrBr",
    bins=8,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Population",
).add_to(m)

folium.LayerControl().add_to(m)
m